<h1> MODEL IMPLEMENTATIONS <h1>

<h2> Imports <h2>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cupy as cp
import sklearn

<h2> Import and Split <h2>

In [3]:
path = r"Dataset\articles.csv"
data = pd.read_csv(path)
print(data.head())

   id                                              title  \
0   0  اریان خان نے والد کی سب سے بڑی خوبی بتا دی انٹ...   
1   1  اداکارہ صحیفہ جبار نے تیزی سے اپنا وزن کیسے کم...   
2   2  کرائم پٹرول کے نوجوان اداکار کی اچانک موت بھار...   
3   3  دیپیکا کی بیٹی کے ہمراہ پہلی جھلک سوشل میڈیا پ...   
4   4  سدھارتھ ملہوترا ماروتی سوزوکی کی مقبول گاڑی ڈز...   

                                                link  \
0  https://www.express.pk/story/2732283/arian-kha...   
1  https://www.express.pk/story/2732277/adakara-s...   
2  https://www.express.pk/story/2732263/crime-pet...   
3  https://www.express.pk/story/2732253/deepika-k...   
4  https://www.express.pk/story/2732223/sidharthm...   

                                             content     gold_label  \
0  بالی ووڈ سپر اسٹار شاہ رخ خان بڑے بیٹے اریان خ...  entertainment   
1  پاکستان شوبز خوبرو اداکارہ ماڈل صحیفہ جبار خٹک...  entertainment   
2  بھارت ٹیلی ویژن اداکار نیتن چوہان گھر سے لاش ب...  entertainment   
3  ممبئی بالی وڈ م

In [105]:
X = data['content']
Y = data['gold_label']

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Y_train shape: {Y_train.shape}")
print(f"Y_test shape: {Y_test.shape}")

import torch

category_to_label = {
    "entertainment": 0,
    "business": 1,
    "sports": 2,
    "science-technology": 3,
    "international": 4
}

Y_train = [category_to_label[label.strip()] for label in Y_train if label.strip() in category_to_label]
Y_test = [category_to_label[label.strip()] for label in Y_test if label.strip() in category_to_label]


X_train shape: (1594,)
X_test shape: (399,)
Y_train shape: (1594,)
Y_test shape: (399,)


<h2> Bag of Words <h2>


In [106]:
class TextProcessor:
    def __init__(self, dataset):
        self.dataset = dataset
        self.vocabulary = {}
    
    def build_vocabulary(self):
        unique_words = set()
        for sentence in self.dataset:
            unique_words.update(sentence.split())
        self.vocabulary = {word: idx for idx, word in enumerate(sorted(unique_words))}
        return self.vocabulary
    
    def sentence_to_bow(self, sentence):
        bow_vector = [0] * len(self.vocabulary)
        for word in sentence.split():
            if word in self.vocabulary:
                bow_vector[self.vocabulary[word]] += 1
        return bow_vector
    
    def vectorize_sentences(self, X):
        return [self.sentence_to_bow(sentence) for sentence in X]


In [ ]:
bag = TextProcessor(X_train)
vocab = bag.build_vocabulary()
train_x = bag.vectorize_sentences(X_train)
test_x = bag.vectorize_sentences(X_test)
# print(vocab)

{'Dtapکہتے': 0, 'Essenceکی': 1, 'FARAکے': 2, 'FDIکو': 3, 'FrançoisPhilippe': 4, 'Graduatesبیروزگار': 5, 'Haitianتارکین': 6, 'Intelsatکمپنی': 7, 'Lollywoodsparkoffical_': 8, 'MGکی': 9, 'OTT_framework_': 10, 'POSCOکےسٹیل': 11, 'PTAکا': 12, 'SULTAN超级巨星': 13, 'Sothebyʼs': 14, 'Standardپر': 15, 'Yکیٹیگری': 16, '___aryan___': 17, 'aima_baig_official': 18, 'aishwaryaraibachchan_arb': 19, 'calorieوالی': 20, 'chidananda_s_naik': 21, 'coke_studio': 22, 'jannatmirza_': 23, 'karpe_offisiell': 24, 'lollywoodsparkofficial_': 25, 'm_koirala': 26, 'natasastankovic_': 27, 'sidhu_moosewala': 28, 'xاور': 29, 'zainoo_': 30, 'ʼحیران': 31, 'ء': 32, 'ءمیں': 33, 'آ': 34, 'آؤٹ': 35, 'آئرلینڈ': 36, 'آئش': 37, 'آئل': 38, 'آئلز': 39, 'آئن': 40, 'آئند': 41, 'آئندہ': 42, 'آئیاکتوبر': 43, 'آئیسولیشن': 44, 'آئیسکو': 45, 'آئین': 46, 'آئیند': 47, 'آئینہ': 48, 'آئینی': 49, 'آئیڈنیٹی': 50, 'آئیڈیاز': 51, 'آئیڈیل': 52, 'آئیڈینٹی': 53, 'آئیکون': 54, 'آئیکی': 55, 'آئیں': 56, 'آئیے': 57, 'آئےانہوں': 58, 'آب': 59, 'آبا': 60, 

<h2> Neural Network Using Pytorch <h2>

In [108]:
import torch
from torch.utils.data import DataLoader, TensorDataset
print(set(Y_train))

train_x = torch.tensor(train_x, dtype=torch.float32)
test_x = torch.tensor(test_x, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.long)
Y_test = torch.tensor(Y_test, dtype=torch.long)

batch_size = 32
train_dataset = TensorDataset(train_x, Y_train)
test_dataset = TensorDataset(test_x, Y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

{0, 1, 2, 3, 4}


In [116]:
import torch.nn as nn
import torch.nn.functional as F

class NewsClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NewsClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 128) 
        self.fc4 = nn.Linear(128, num_classes) 

    def forward(self, x):
        x = F.relu(self.fc1(x))   
        x = self.fc4(x)          
        return x


In [117]:
input_size = train_x.shape[1]
num_classes = len(Y_train.unique())
model = NewsClassifier(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [118]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch_x, batch_y in train_loader:
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")


Epoch [1/5], Loss: 0.6065
Epoch [2/5], Loss: 0.0594
Epoch [3/5], Loss: 0.0198
Epoch [4/5], Loss: 0.0101
Epoch [5/5], Loss: 0.0065


In [119]:
# Evaluation function
def evaluate_model(loader, model):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_x, batch_y in loader:
            outputs = model(batch_x)
            _, predicted = torch.max(outputs, 1)  
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()
    
    accuracy = correct / total
    return accuracy

train_accuracy = evaluate_model(train_loader, model)
test_accuracy = evaluate_model(test_loader, model)
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.2f}")


Training Accuracy: 1.0000
Test Accuracy: 0.97


In [124]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Assuming X_train, X_test, Y_train, Y_test are already defined and preprocessed

# Standardizing the features (Logistic Regression usually benefits from feature scaling)
scaler = StandardScaler()
X_train_scaled = train_x   # Fit and transform on training data
X_test_scaled = test_x       # Only transform on test data

# Initialize and train the Logistic Regression model
model = LogisticRegression(max_iter=1000, multi_class='ovr', solver='liblinear')  # Solver 'liblinear' is good for smaller datasets
model.fit(X_train_scaled, Y_train)

# Make predictions
train_predictions = model.predict(X_train_scaled)
test_predictions = model.predict(X_test_scaled)

# Calculate accuracy
train_accuracy = accuracy_score(Y_train, train_predictions)
test_accuracy = accuracy_score(Y_test, test_predictions)

# Print accuracies
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


Training Accuracy: 1.0000
Test Accuracy: 0.9649


In [126]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
model_nb = MultinomialNB()

# Fit the model on the training data
model_nb.fit(X_train_scaled, Y_train)

# Make predictions on both train and test datasets
train_predictions_nb = model_nb.predict(X_train_scaled)
test_predictions_nb = model_nb.predict(X_test_scaled)

# Calculate accuracy
train_accuracy_nb = accuracy_score(Y_train, train_predictions_nb)
test_accuracy_nb = accuracy_score(Y_test, test_predictions_nb)

# Print accuracies
print(f"Training Accuracy (Naive Bayes): {train_accuracy_nb:.4f}")
print(f"Test Accuracy (Naive Bayes): {test_accuracy_nb:.4f}")

Training Accuracy (Naive Bayes): 0.9806
Test Accuracy (Naive Bayes): 0.9674
